In [1]:
### INIT Pyside and ScriptEdit ###

from PySide6.QtWidgets import *
%gui qt
app = QApplication.instance()  # Reuse the existing instance if available
if app is None:  # If no instance exists, create one
    app = QApplication([])

# Create a simple Qt widget
from pylive.QtScriptEditor.script_edit import ScriptEdit
script_edit = ScriptEdit()
script_edit.show()

setTabSize 4


In [2]:
# set the script
from textwrap import dedent
script_edit.setPlainText(dedent("""\
def hello():
    print("boom")
"""))

In [ ]:
# from pylive.QtScriptEditor.components.invisibles_highlighter import InvisiblesHighlighter

from pylive.QtScriptEditor.components.simple_python_highlighter import SimplePythonHighlighter
script_edit.highlighter = SimplePythonHighlighter(script_edit.document())

In [7]:
from pylive.QtScriptEditor.components.pygments_syntax_highlighter import PygmentsSyntaxHighlighter
script_edit.highlighter = PygmentsSyntaxHighlighter(script_edit.document())

In [ ]:
import sys
from PySide6.QtWidgets import *
from PySide6.QtGui import *
from PySide6.QtCore import *

from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatter import Formatter
from pygments.token import Token

def complement(color:QColor):
    h, s, l, a = color.getHsl()
    return QColor.fromHsl((h+180)%360, s, l, a)

def shift(color:QColor, offset=180):
    h, s, l, a = color.getHsl()
    return QColor.fromHsl((h+offset)%360, s, l, a)

def dim(color:QColor, a=128):
    r,g,b,_ = color.getRgb()
    return QColor.fromRgb( r,g,b,a )
dim

class PygmentsSyntaxHighlighter(QSyntaxHighlighter):
    def __init__(self, document):
        super().__init__(document)
        self.lexer = PythonLexer()  # Use the lexer for the language you want to highlight

    def highlightBlock(self, text):
        """ Apply syntax highlighting to a block of text. """
        # Create the formatter for this block
        formatter = QtFormatter()
        # Use Pygments to tokenize the text and apply formatting
        highlight(text, self.lexer, formatter)

        # Apply the formats calculated by the formatter
        for start, length, format in formatter.formats:
            self.setFormat(start, length, format)
import re
class QtFormatter(Formatter):
    """ Formatter that applies Pygments tokens to QTextCharFormat
    for QSyntaxHighlighter """
    def __init__(self):
        super().__init__()
        self.formats = []

    def format(self, tokensource, outfile):
        """ Convert Pygments tokens into QTextCharFormat for
        QSyntaxHighlighter """
        palette = QGuiApplication.palette()
        current_pos = 0
        for ttype, value in tokensource:
            length = len(value)
            if length > 0:
                # Create a QTextCharFormat based on the token type
                fmt = QTextCharFormat()

                # Apply basic styles from Pygments tokens
                if 'Keyword' in str(ttype):
                    # Blue for keywords
                    fmt.setForeground(complement(palette.color(QPalette.ColorRole.Highlight)))
                elif 'String' in str(ttype):
                    # Green for strings
                    fmt.setForeground(QColor("yellow"))
                elif 'Comment' in str(ttype):
                    # Gray for comments
                    fmt.setForeground(dim( palette.color(QPalette.ColorRole.Text), 150 )) 
                    fmt.setFontItalic(True)
                elif 'Name.Function' in str(ttype):
                    # Purple for function names
                    fmt.setForeground(QColor("darkcyan"))
                    fmt.setFontWeight(QFont.Bold)

                elif 'Number' in str(ttype):
                    fmt.setForeground(QColor("green"))

                elif ttype == Token.Operator:
                    fmt.setForeground(QColor("green"))

                # Whitespace
                elif re.match(r'[ \t]+', value):
                    text_color = palette.color(QPalette.ColorRole.Text)
                    text_color.setAlpha(30)
                    fmt.setForeground(text_color)
                elif "Text.Whitespace" in str(ttype):
                    text_color = palette.color(QPalette.ColorRole.Text)
                    text_color.setAlpha(100)
                    fmt.setForeground(QColor("darkcyan"))
                else:
                    # print(ttype, value)
                    # Default to black for others
                    fmt.setForeground(palette.color(QPalette.ColorRole.Text))

                # Append the format with the
                # start position and length
                self.formats.append((current_pos, length, fmt))

            # Update the current position
            current_pos += length

script_edit.highlighter = PygmentsSyntaxHighlighter(script_edit.document())


palette = script_edit.palette()
palette.setBrush(QPalette.Base, QBrush(QColor("rgb(48, 56, 65)")))  # Set the Base color for the text edit
script_edit.setPalette(palette)